In [1]:
import joblib
import numpy as np
import textstat
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
from textstat import flesch_reading_ease, gunning_fog
from textblob import TextBlob
import datetime
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

rf_model = joblib.load('./model/random_forest_model.joblib')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
def main():
    # User inputs
    score = input("Enter the score: ")
    score = int(score)
    summary = input("Enter the summary: ")
    text = input("Enter the text: ")

    # Calculations
    text_length = minmax_scale_input(len(text),19,25738)
    summary_length = minmax_scale_input(len(summary),0,128)
    
    # You can fill in the functions to calculate these values
    Vader_text = VaderPrediction(text)
    Vader_sum = VaderPrediction(summary)
    text_neg = Vader_text['neg']
    text_neu = Vader_text['neu']
    text_pos = Vader_text['pos']
    text_compound_score = Vader_text['compound']
    sum_neg = Vader_sum['neg']
    sum_neu = Vader_sum['neu']
    sum_pos = Vader_sum['pos']
    sum_compound_score = Vader_sum['compound']
    
    
    flesch_reading_ease = minmax_scale_input(textstat.flesch_reading_ease(text),-257.73,119.19)
    gunning_fog = minmax_scale_input(textstat.gunning_fog(text),1.2,138.39)
    avg_word_length_summary = minmax_scale_input(average_word_length(summary),0,114)
    avg_word_length_text = minmax_scale_input(average_word_length(text),2,26)
    subjectivity_summary = calculate_subjectivity(summary)
    subjectivity_text = calculate_subjectivity(text)
    ttr_text = type_token_ratio(text)
    if_weekday_1 = is_weekday()
    if_weekday_0 = switch_input(if_weekday_1)
    features = [
    score, text_length, summary_length, text_neg, text_neu, text_pos,
    text_compound_score, sum_neg, sum_neu, sum_pos, sum_compound_score,
    flesch_reading_ease, gunning_fog, avg_word_length_summary,
    avg_word_length_text, subjectivity_summary, subjectivity_text, 
    ttr_text, if_weekday_0, if_weekday_1
]

    X = np.array(features).reshape(1, -1)
    predictions = rf_model.predict(X)
    if predictions == 0:
        print('Model Prediction: This review might not be helpful...')
    else:
        print('Model Prediction: This review is likely to be helpful!')
        
        
    return 0

    
def minmax_scale_input(input_data, min_val, max_val):
    return (input_data - min_val) / (max_val - min_val)

def average_word_length(text):
    words = text.split()
    if len(words) == 0:
        return 0
    return sum(len(word) for word in words) / len(words)

def calculate_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def type_token_ratio(text):
    tokens = text.split()
    types = set(tokens)
    if len(tokens) == 0:
        return 0  # Avoid division by zero
    return len(types) / len(tokens)

def is_weekday():
    today = datetime.datetime.now()
    return 1 if 0 <= today.weekday() < 5 else 0

def switch_input(input_value):
    return 1 if input_value == 0 else 0

def VaderPrediction(text):
    return sia.polarity_scores(text)    

In [3]:
if __name__ == "__main__":
    main()

Enter the score: 2
Enter the summary: what
Enter the text: I ate a sandwich tonight and it was good.
Model Prediction: This review might not be helpful...


In [4]:
if __name__ == "__main__":
    main()

Enter the score: 5
Enter the summary:  "The Timeless Journey" is a visually stunning science fiction drama that explores the themes of love, loss, and the human connection across time. Set in a future where time travel is possible, the film follows the story of Ella, a scientist who embarks on a journey to the past to prevent a global catastrophe. Along the way, she encounters versions of her loved ones, leading to poignant reflections on her life choices and relationships.
Enter the text:  This film is a mesmerizing blend of thought-provoking narrative and breathtaking cinematography. The director skillfully intertwines complex scientific concepts with deeply human emotions, making the story both intellectually stimulating and emotionally resonant. The lead, played by Alex Rivera, delivers a powerful performance, capturing the essence of a person torn between her professional mission and personal desires. The visual effects are top-notch, creating a believable yet fantastical world of